In [26]:
import os
import time
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import regex as re
import requests
import plotly.express as px
import plotly.graph_objects as go
import json
import datetime
from scipy import stats
import tqdm


In [37]:
#Change the city to the city you want to pull data for
city = 'ocean-gate-nj'
#city = 'forked-river-nj'

In [38]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

df1 = pd.DataFrame()
base_url = 'https://www.zillow.com/'+ city +'/sold/'  

def find_list_results(data):
    if isinstance(data, dict):
        for k, v in data.items():
            if k == 'listResults':
                return v
            else:
                result = find_list_results(v)
                if result:
                    return result
    elif isinstance(data, list):
        for item in data:
            result = find_list_results(item)
            if result:
                return result
    return None

for beds_count in [1, 2, 3, 4, 5]:
    for baths_count in [1.0, 1.5, 2.0, 3.0, 4.0]:
        #Loop through all 20 of zillows pages
        for i in tqdm.tqdm(range(1,20)):
            with requests.Session() as s:

                #sleep so that they think we're human
                time.sleep(np.random.randint(1,3))
                
                if i == 1:
                    page = ""
                else: 
                    page = str(i) + "_p/"

                bed_part =  str(beds_count) + "-_beds/"
                bath_part = str(baths_count) + "-_baths/"
                
                url = base_url + bed_part + bath_part + page
                
                patterns = [
                    r'<script id="__NEXT_DATA__" type="application/json">(.*?)</script>',
                    r'({"props":{"encodedZuid":"",.*?})',
                    r'<script type="application/json" data-zrr-shared-data-key="mobileSearchPageStore" ><!--(.*?)-->',
                ]

                r = s.get(url, headers=req_headers)

                match = None
                for pattern in patterns:
                    match = re.search(pattern, r.text, re.DOTALL)
                    if match:
                        break

                if match is None:
                    print(f"No match found on page {i} for beds: {beds_count}, baths: {baths_count}")
                else:
                    data_string = match.group(1)
                    try:
                        data = json.loads(data_string)
                    except json.JSONDecodeError as e:
                        print(f"Failed to parse JSON on page {i} for beds: {baths_count}, baths: {baths_count}: {e}")
                        print(f"Offending string: {data_string}")
                        continue

                    try:
                        list_results = find_list_results(data)
                        if list_results is None:
                            print(f"'listResults' not found on page {i} for beds: {beds_count}, baths: {baths_count}")
                            continue
                        df = pd.json_normalize(list_results)

                        # store in a dataframe
                        df1 = df1.append(df)
                    except Exception as e:
                        print(f"Failed to create DataFrame on page {i} for beds: {beds_count}, baths: {baths_count}: {e}")


  0%|          | 0/19 [00:00<?, ?it/s]

100%|██████████| 19/19 [00:40<00:00,  2.13s/it]


In [39]:
#Drop duplicates
df = df1.drop_duplicates(subset=['zpid'], keep='first')
df = df.reset_index(drop=True)  

In [40]:
##Save CSV with today's date
today = time.strftime("%Y%m%d")
df.to_csv('results/'+city+'_'+today+'.csv')

In [44]:
#Pull in all past data for analysis
files = [f for f in os.listdir('results/') if f.endswith('.csv')]

#Only keep files with city variable in them
files = [f for f in files if city in f]

#Combine all CSV files into one
df = pd.concat([pd.read_csv('results/'+ f) for f in files])

#Remove duplicates
df = df.drop_duplicates(subset=['zpid'], keep='first')
df = df.reset_index(drop=True)

### Analyse price per sqft of single family homes

In [45]:
######Analyse price per sqft of single family homes ######

#Only single family homes
df = df[df['hdpData.homeInfo.homeType'] == 'SINGLE_FAMILY']

#Subset data to drop rows with NAN in area
df = df[pd.notnull(df['area'])]

#sold_date to date
df['sold_date'] = pd.to_datetime(df['hdpData.homeInfo.dateSold'], unit='ms')

#Convert to int
df['price'] = df['unformattedPrice'].astype(int)
df['sqft'] = df['area'].astype(int)
df['beds'] = df['beds'].astype(int)
df['baths'] = df['baths'].astype(int)

#Remove Crazy outliers
df['sqft_zscore'] = stats.zscore(df['sqft'])
df = df[np.abs(df['sqft_zscore']) < 2]


#calculate the average price per sqft
df['price_sqft'] = df['price'] / df['sqft']

#create plotly scatter chart of avg price per sqft over time
fig = px.scatter(df, x='sold_date', y='price_sqft', color ='price',  
                color_continuous_scale=px.colors.sequential.Turbo, range_color=(100000, 1000000), title='Price Per Square Foot Over Time in '+city)
fig.update_traces(marker={'size': 5, 'opacity': 0.75})


#### Now group the data by week and create an EMA with an 8 week window

#Group data by day sold
df2 = df.groupby(['sold_date'], as_index=False)["price_sqft"].mean()

#add price ema
df2['price_ema'] = df2['price_sqft'].ewm(span=30).mean()

##Add trace to plotly fig
fig.add_traces(
    list(px.line(df2, x='sold_date', y='price_ema').select_traces())
    )
fig.update_traces(line_color='#303030', line_width=5)
fig.show()


### Analyse the taxEstimate to sold price ratio

In [47]:
###### Analyse the taxEstimate to sold price ratio  ######
df2 = df[pd.notnull(df['hdpData.homeInfo.taxAssessedValue'])]

#Take only the homes priced 275k to 700k
df2 = df2[(df2['price'] > 275000) & (df2['price'] < 700000)]

#Create a new column with the taxEstimate to sold price ratio
df2['sptv_ratio'] = df2['price'] / df2['hdpData.homeInfo.taxAssessedValue']

#Remove major outliers
df2['sptv_zscore'] = stats.zscore(df2['sptv_ratio'])
df2 = df2[np.abs(df2['sptv_zscore']) < 1]

##### Plot Sold Price to Tax Estimate Ratio #####
fig = px.scatter(df2, x='sold_date', y='sptv_ratio', color ='price',  
                 color_continuous_scale=px.colors.sequential.Turbo, title='Sold Price to Tax Accessed Value over time in '+city)
fig.update_traces(marker={'size': 5, 'opacity': 0.75})

### Add moving EMA by week
##floor date by week
df3 = df2.groupby(['sold_date'], as_index=False)["sptv_ratio"].mean()

#add sptv_ratio ema
df3['sptv_ratio_ema'] = df3['sptv_ratio'].ewm(span=30).mean()

##Add EMA trace to plotly fig
fig.add_trace(go.Scatter(x=df3['sold_date'], y= df3['sptv_ratio_ema'], mode='lines', line_color='#303030', line_width=4))

fig.data = (fig.data[1],fig.data[0])
fig.show()

